%%latex

Simulating a DDM means simulating the noisy trajectories that the Decision Variable (DV) follows from the starting point until they reach one of the thresholds, which determines the Reaction Time (RT, the time at which the threshold was hit) and the choice (which threshold was hit). Mathematically, the sub-threshold evolution of the decision variable $x$ is described by the following dynamics (in what's called the Langevin Equation) 

\begin{equation}
    dx = \nu dt + \sigma dB_t \quad x(t=0) = x_0
\end{equation}
The bounds are located at $\pm \theta$. The parameters $\nu$ and $\sigma^2$ measure, respectively, the mean and variance of the sensory evidence. What this equation is saying, intuitively, is that at every small increment of time $dt$, the decision variable increases by a constant term $\nu dt$, where $\nu$ is called the "drift" of the system (also often referred to as the strength of evidence), plus a stochastic term $\sigma dB_t$. The quantity $dBt$ can be understood as a normal (gaussian) random variable with mean zero and variance $dt$. This implies that its standard deviation is $\sqrt{dt}$, and so, because $dt$ is very small, it's going to much larger than the drift term. However, the drift term is consistent, while the noise term isn't (it has mean zero), so on average $x$ will evolve in the direction of the sign of $\nu$, and faster and more deterministicly the stronger $\nu$ is. 
Note that because the increment of $x$ does not depend on $x$, the `zero' value of the decision variable is arbitrary. That is why we can always choose the bounds as symmetric. 

It turns out that this system is overparametrized, meaning that you can find combinations of parameters that give you exactly the same distributions of RTs and choices. A common way to adress this is to remove $\sigma$ by normalizing the DV and the bounds relative to it:
\begin{equation}
    \sigma d\frac{x}{\sigma} = \sigma \frac{\nu}{\sigma} dt + \sigma dB_t 
\end{equation}
Thus, if instead of the original problem, we consider an alternative problem with the decision variable $x'=x/\sigma$ and $v \equiv \nu/\sigma$, we would obtain
\begin{equation}
    dx' = v dt + dB_t 
\end{equation}
with bounds at $x' = \pm \theta/\sigma$ and starting position $x'(t=0) = x_0/\sigma \equiv x'_0$. Since, as we said above, the origin of the decision variable is arbitrary, it is common to put the origin at the lower bound, so we define $\tilde{x} = x' - (-\theta/\sigma)$. Because this is just a shift in offset, the dynamical equation doesn't change
\begin{equation}
    d\tilde{x} = v dt + dB_t 
\end{equation}
the bounds for this problem are at $\tilde{x}=0$ and $2\theta/\sigma \equiv a$, and the initial condition is at $\tilde{x}(t=0) = x'_0+\theta/\sigma \equiv \tilde{x}_0$, which can vary between 0 and $a$. It is customary to normalize the initial condition in units of $a$, so that instead of $\tilde{x}_0$ one uses $w \equiv \tilde{x}_0/a$, which can take any value between 0 and 1. This version of the model, with parameters $\{v,w,a\}$, and is considered the "standard model" in the literature.



%%latex

That was some math! But don't worry, when you play with it you will see that it's not scary.
The first task will be to simulate this simple DDM. You just need to translate the math describing the evolution of the DV into code. 

Write a simple simulator function that takes as input the parameters $\{v,w,a\}$ and gives back $N$ choices and RT

In [1]:
import numpy as np

def simulator(v,w,a,N):

    # think of how to initialize things

    for k in range(N):
        # while bound has not been hit

        x[k,t_i] = x_[k,t_i-1] + ...

    choices[k] = ...
    RTs[k] = ...

    return choices, RTs, x

After you have the simulator, there's many things you can plot: RT distributions, psychometric functions, chronometric functions... I'll tell you more about it when you get there!